In [7]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import warnings

In [9]:
hr = pd.read_csv('HR_comma_sep.csv')

In [11]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14995 entries, 0 to 14994
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14995 non-null  float64
 1   last_evaluation        14995 non-null  float64
 2   number_project         14995 non-null  int64  
 3   average_montly_hours   14995 non-null  int64  
 4   time_spend_company     14995 non-null  int64  
 5   Work_accident          14995 non-null  int64  
 6   left                   14995 non-null  int64  
 7   promotion_last_5years  14995 non-null  int64  
 8   Department             14995 non-null  object 
 9   salary                 14995 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [13]:
X = hr.drop('left',axis=1)
y = hr['left']

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

## catboost Without OHE

In [31]:
cgbm = CatBoostClassifier(random_state=24, cat_features=['Department','salary'],num_boost_round = 50)
cgbm.fit(X_train,y_train)

Learning rate set to 0.438486
0:	learn: 0.4127012	total: 40.8ms	remaining: 2s
1:	learn: 0.2883445	total: 81.8ms	remaining: 1.96s
2:	learn: 0.2171350	total: 123ms	remaining: 1.93s
3:	learn: 0.1787252	total: 170ms	remaining: 1.95s
4:	learn: 0.1554308	total: 215ms	remaining: 1.94s
5:	learn: 0.1402544	total: 262ms	remaining: 1.92s
6:	learn: 0.1274649	total: 307ms	remaining: 1.89s
7:	learn: 0.1223568	total: 352ms	remaining: 1.85s
8:	learn: 0.1151863	total: 396ms	remaining: 1.8s
9:	learn: 0.1090791	total: 440ms	remaining: 1.76s
10:	learn: 0.1055714	total: 486ms	remaining: 1.72s
11:	learn: 0.1027904	total: 529ms	remaining: 1.67s
12:	learn: 0.1001674	total: 571ms	remaining: 1.63s
13:	learn: 0.0980211	total: 613ms	remaining: 1.58s
14:	learn: 0.0960815	total: 656ms	remaining: 1.53s
15:	learn: 0.0946345	total: 702ms	remaining: 1.49s
16:	learn: 0.0927250	total: 744ms	remaining: 1.45s
17:	learn: 0.0906349	total: 786ms	remaining: 1.4s
18:	learn: 0.0885028	total: 829ms	remaining: 1.35s
19:	learn: 0.0

In [35]:
y_pred = cgbm.predict(X_test)

In [37]:
accuracy_score(y_test,y_pred)

0.9773282951767059

In [39]:
roc_auc_score(y_test,y_pred)

0.9603733139276593

In [41]:
cgbm.get_params()

{'num_boost_round': 50,
 'random_state': 24,
 'cat_features': ['Department', 'salary']}

In [57]:
cgbm = CatBoostClassifier(random_state=24, cat_features=['Department','salary'])

In [71]:
params = {
         'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
          
          'subsample': np.linspace(0,1,10)
         }
          
          
         

kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state =24)

In [73]:
gcv = GridSearchCV(cgbm,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
90 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  Fil

Learning rate set to 0.143225
0:	learn: 0.5077584	total: 39ms	remaining: 7.75s
1:	learn: 0.3964866	total: 78.8ms	remaining: 7.8s
2:	learn: 0.3146199	total: 130ms	remaining: 8.55s
3:	learn: 0.2781299	total: 171ms	remaining: 8.37s
4:	learn: 0.2403384	total: 224ms	remaining: 8.73s
5:	learn: 0.2189586	total: 268ms	remaining: 8.66s
6:	learn: 0.1977221	total: 313ms	remaining: 8.62s
7:	learn: 0.1866587	total: 361ms	remaining: 8.65s
8:	learn: 0.1799494	total: 412ms	remaining: 8.74s
9:	learn: 0.1673161	total: 461ms	remaining: 8.77s
10:	learn: 0.1585461	total: 509ms	remaining: 8.75s
11:	learn: 0.1517310	total: 558ms	remaining: 8.74s
12:	learn: 0.1455445	total: 604ms	remaining: 8.68s
13:	learn: 0.1409792	total: 651ms	remaining: 8.64s
14:	learn: 0.1347240	total: 695ms	remaining: 8.57s
15:	learn: 0.1313214	total: 738ms	remaining: 8.49s
16:	learn: 0.1276620	total: 780ms	remaining: 8.4s
17:	learn: 0.1247629	total: 822ms	remaining: 8.31s
18:	learn: 0.1215089	total: 866ms	remaining: 8.25s
19:	learn: 0.

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x000001E6DCD7AA20>,
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4],
                         'n_estimators': [10, 20, 30, 50, 100, 200],
                         'subsample': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])},
             scoring='roc_auc', verbose=3)

In [75]:
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': 4, 'n_estimators': 200, 'subsample': 0.6666666666666666}
0.9916761414671527
